In [ ]:
import sqlite3
from datetime import datetime


class Database:
    def __init__(self, db_name="todos.db"):
        self.conn = sqlite3.connect(db_name)
        self.create_tables()

    def create_tables(self):
        c = self.conn.cursor()
        c.execute('''
            CREATE TABLE IF NOT EXISTS users (
                id INTEGER PRIMARY KEY AUTOINCREMENT,
                username TEXT UNIQUE NOT NULL
            )
        ''')
        c.execute('''
            CREATE TABLE IF NOT EXISTS todos (
                id INTEGER PRIMARY KEY AUTOINCREMENT,
                user_id INTEGER,
                task TEXT NOT NULL,
                description TEXT,
                priority TEXT CHECK(priority IN ('Low', 'Medium', 'High')) DEFAULT 'Medium',
                category TEXT CHECK(category IN ('Work', 'Personal', 'Other')) DEFAULT 'Other',
                due_date TEXT,
                done BOOLEAN DEFAULT 0,
                FOREIGN KEY (user_id) REFERENCES users(id)
            )
        ''')
        self.conn.commit()

    def execute(self, query, params=(), fetch=False):
        c = self.conn.cursor()
        c.execute(query, params)
        self.conn.commit()
        if fetch:
            return c.fetchall()

    def close(self):
        self.conn.close()


class User:
    def __init__(self, db: Database):
        self.db = db
        self.user_id = None

    def register(self, username):
        try:
            self.db.execute("INSERT INTO users (username) VALUES (?)", (username,))
            print(f"User '{username}' registered successfully.")
        except sqlite3.IntegrityError:
            print("Username already exists.")

    def login(self, username):
        result = self.db.execute("SELECT id FROM users WHERE username = ?", (username,), fetch=True)
        if result:
            self.user_id = result[0][0]
            print(f"Logged in as '{username}'")
            return True
        else:
            print("User not found. Please register.")
            return False


class Todo:
    def __init__(self, db: Database, user: User):
        self.db = db
        self.user = user
        self.task_map = {}

    def get_nonempty_input(self, prompt):
        while True:
            value = input(prompt).strip()
            if value:
                return value
            print("This field is compulsory. Please enter a valid value.")

    def get_valid_due_date(self, prompt):
        while True:
            due_date = input(prompt).strip()
            try:
                datetime.strptime(due_date, "%Y-%m-%d %H:%M")
                return due_date
            except ValueError:
                print("Invalid date format. Please enter date and time as YYYY-MM-DD HH:MM.")

    def add_task(self, task=None):
        if not task:
            task = self.get_nonempty_input("Enter task title (compulsory): ")
        else:
            if not task.strip():
                task = self.get_nonempty_input("Enter task title (compulsory): ")

        description = input("Enter description (optional): ")

        priority = input("Enter priority (High, Medium, Low) [default Medium]: ").capitalize()
        if priority not in ["High", "Medium", "Low"]:
            priority = "Medium"

        category = input("Enter category (Work, Personal, Other) [default Other]: ").capitalize()
        if category not in ["Work", "Personal", "Other"]:
            category = "Other"

        due_date = self.get_valid_due_date("Enter due date and time (YYYY-MM-DD HH:MM) (compulsory): ")

        self.db.execute(
            "INSERT INTO todos (user_id, task, description, priority, category, due_date) VALUES (?, ?, ?, ?, ?, ?)",
            (self.user.user_id, task, description, priority, category, due_date)
        )
        print("Task added.")

    def view_tasks(self):
        tasks = self.db.execute(
            "SELECT id, task, description, priority, category, due_date, done FROM todos WHERE user_id = ?",
            (self.user.user_id,),
            fetch=True
        )
        self.task_map = {}
        if not tasks:
            print("No tasks.")
            return

        for i, t in enumerate(tasks, start=1):
            self.task_map[i] = t[0]
            status = "✅" if t[6] else "❌"
            due_str = t[5]

            overdue = False
            if due_str:
                try:
                    due_date = datetime.strptime(due_str, "%Y-%m-%d %H:%M")
                    if not t[6] and datetime.now() > due_date:
                        overdue = True
                except ValueError:
                    print(f"⚠️ Invalid due date format for task {i}")

            if overdue:
                print(f"\033[91m{i}. {t[1]} [{t[3]} | {t[4]}] Due: {due_str} {status}\n   ➤ {t[2]}\033[0m")
            else:
                print(f"{i}. {t[1]} [{t[3]} | {t[4]}] Due: {due_str} {status}\n   ➤ {t[2]}")

    def mark_done(self, local_id):
        real_id = self.task_map.get(local_id)
        if real_id:
            self.db.execute("UPDATE todos SET done = 1 WHERE id = ? AND user_id = ?", (real_id, self.user.user_id))
            print("Task marked as done.")
        else:
            print("Invalid task number.")

    def delete_task(self, local_id):
        real_id = self.task_map.get(local_id)
        if real_id:
            self.db.execute("DELETE FROM todos WHERE id = ? AND user_id = ?", (real_id, self.user.user_id))
            print("Task deleted.")
        else:
            print("Invalid task number.")

    def update_task(self, local_id, new_text=None):
        real_id = self.task_map.get(local_id)
        if real_id:
            if not new_text:
                new_text = self.get_nonempty_input("Enter new task title (compulsory): ")
            else:
                if not new_text.strip():
                    new_text = self.get_nonempty_input("Enter new task title (compulsory): ")

            new_description = input("Enter new description (optional): ")

            new_priority = input("Enter new priority (High, Medium, Low) [leave blank to keep current]: ").capitalize()
            if new_priority not in ["High", "Medium", "Low"]:
                result = self.db.execute("SELECT priority FROM todos WHERE id = ? AND user_id = ?", (real_id, self.user.user_id), fetch=True)
                if result:
                    new_priority = result[0][0]

            new_category = input("Enter new category (Work, Personal, Other) [leave blank to keep current]: ").capitalize()
            if new_category not in ["Work", "Personal", "Other"]:
                result = self.db.execute("SELECT category FROM todos WHERE id = ? AND user_id = ?", (real_id, self.user.user_id), fetch=True)
                if result:
                    new_category = result[0][0]

            new_due_date = self.get_valid_due_date("Enter new due date and time (YYYY-MM-DD HH:MM) (compulsory): ")

            self.db.execute(
                "UPDATE todos SET task = ?, description = ?, priority = ?, category = ?, due_date = ? WHERE id = ? AND user_id = ?",
                (new_text, new_description, new_priority, new_category, new_due_date, real_id, self.user.user_id)
            )
            print("Task updated.")
        else:
            print("Invalid task number.")

    def filter_by_specific_date(self):
        date_str = input("Enter date (YYYY-MM-DD): ")
        try:
            datetime.strptime(date_str, "%Y-%m-%d")
        except ValueError:
            print("Invalid date format.")
            return
    
        query = """
            SELECT id, task, description, priority, category, due_date, done 
            FROM todos 
            WHERE user_id = ? AND DATE(due_date) = ?
        """
        tasks = self.db.execute(query, (self.user.user_id, date_str), fetch=True)
    
        self.task_map = {}
        if not tasks:
            print("No tasks due on this date.")
            return
    
        for i, t in enumerate(tasks, start=1):
            self.task_map[i] = t[0]
            status = "✅" if t[6] else "❌"
            overdue = False
            if t[5]:
                try:
                    due_date = datetime.strptime(t[5], "%Y-%m-%d %H:%M")
                    if not t[6] and datetime.now() > due_date:
                        overdue = True
                except:
                    pass
    
            if overdue:
                print(f"\033[91m{i}. {t[1]} [{t[3]} | {t[4]}] Due: {t[5]} {status}\n   ➤ {t[2]}\033[0m")
            else:
                print(f"{i}. {t[1]} [{t[3]} | {t[4]}] Due: {t[5]} {status}\n   ➤ {t[2]}")
    
    def sort_tasks(self):
        print("\nSort by:\n1. Due Date\n2. Priority")
        choice = input("Choose option (1 or 2): ")
    
        if choice == "1":
            query = """
                SELECT id, task, description, priority, category, due_date, done 
                FROM todos 
                WHERE user_id = ? 
                ORDER BY datetime(due_date) ASC
            """
        elif choice == "2":
            query = """
                SELECT id, task, description, priority, category, due_date, done 
                FROM todos 
                WHERE user_id = ?
                ORDER BY CASE 
                    WHEN priority = 'High' THEN 1 
                    WHEN priority = 'Medium' THEN 2 
                    WHEN priority = 'Low' THEN 3 
                    ELSE 4 
                END
            """
        else:
            print("Invalid choice.")
            return

        tasks = self.db.execute(query, (self.user.user_id,), fetch=True)
        self.task_map = {}
    
        if not tasks:
            print("No tasks to display.")
            return
    
        for i, t in enumerate(tasks, start=1):
            self.task_map[i] = t[0]
            status = "✅" if t[6] else "❌"
            overdue = False
            if t[5]:
                try:
                    due_date = datetime.strptime(t[5], "%Y-%m-%d %H:%M")
                    if not t[6] and datetime.now() > due_date:
                        overdue = True
                except:
                    pass
    
            if overdue:
                print(f"\033[91m{i}. {t[1]} [{t[3]} | {t[4]}] Due: {t[5]} {status}\n   ➤ {t[2]}\033[0m")
            else:
                print(f"{i}. {t[1]} [{t[3]} | {t[4]}] Due: {t[5]} {status}\n   ➤ {t[2]}")

    def filter_tasks(self):
        print("\nFilter tasks by:")
        print("1. All Tasks")
        print("2. Done Tasks")
        print("3. Not Done Tasks")
        print("4. Priority (High, Medium, Low)")
        print("5. Category (Work, Personal, Other)")
        
        choice = input("Choose filter option: ")
    
        base_query = "SELECT id, task, description, priority, category, due_date, done FROM todos WHERE user_id = ?"
        params = [self.user.user_id]
    
        if choice == "1":
            # All tasks
            query = base_query
        elif choice == "2":
            # Done tasks only
            query = base_query + " AND done = 1"
        elif choice == "3":
            # Not done tasks only
            query = base_query + " AND done = 0"
        elif choice == "4":
            priority = input("Enter priority to filter by (High, Medium, Low): ").capitalize()
            if priority not in ["High", "Medium", "Low"]:
                print("Invalid priority.")
                return
            query = base_query + " AND priority = ?"
            params.append(priority)
        elif choice == "5":
            category = input("Enter category to filter by (Work, Personal, Other): ").capitalize()
            if category not in ["Work", "Personal", "Other"]:
                print("Invalid category.")
                return
            query = base_query + " AND category = ?"
            params.append(category)
        else:
            print("Invalid choice.")
            return
    
        tasks = self.db.execute(query, tuple(params), fetch=True)
        self.task_map = {}
    
        if not tasks:
            print("No tasks found for this filter.")
            return
    
        for i, t in enumerate(tasks, start=1):
            self.task_map[i] = t[0]
            status = "✅" if t[6] else "❌"
            overdue = False
            if t[5]:
                try:
                    due_date = datetime.strptime(t[5], "%Y-%m-%d %H:%M")
                    if not t[6] and datetime.now() > due_date:
                        overdue = True
                except:
                    pass
    
            if overdue:
                print(f"\033[91m{i}. {t[1]} [{t[3]} | {t[4]}] Due: {t[5]} {status}\n   ➤ {t[2]}\033[0m")
            else:
                print(f"{i}. {t[1]} [{t[3]} | {t[4]}] Due: {t[5]} {status}\n   ➤ {t[2]}")



def main():
    db = Database()
    user = User(db)

    while True:
        print("\n1. Register")
        print("2. Login")
        print("3. Exit")

        choice = input("Choose option: ")
        if choice == "1":
            username = input("Enter username: ").strip()
            user.register(username)
        elif choice == "2":
            username = input("Enter username: ").strip()
            if user.login(username):
                todo = Todo(db, user)
                while True:
                    print("\nOptions:")
                    print("1. Add task")
                    print("2. View tasks")
                    print("3. Mark task done")
                    print("4. Delete task")
                    print("5. Update task")
                    print("6. Filter tasks")
                    print("7. Sort tasks")
                    print("8. Filter by date")
                    print("9. Logout")

                    opt = input("Choose option: ")

                    if opt == "1":
                        todo.add_task()
                    elif opt == "2":
                        todo.view_tasks()
                    elif opt == "3":
                        todo.view_tasks()
                        task_no = int(input("Enter task number to mark done: "))
                        todo.mark_done(task_no)
                    elif opt == "4":
                        todo.view_tasks()
                        task_no = int(input("Enter task number to delete: "))
                        todo.delete_task(task_no)
                    elif opt == "5":
                        todo.view_tasks()
                        task_no = int(input("Enter task number to update: "))
                        todo.update_task(task_no)
                    elif opt == "6":
                        todo.filter_tasks()
                    elif opt == "7":
                        todo.sort_tasks()
                    elif opt == "8":
                        todo.filter_by_specific_date()
                    elif opt == "9":
                        print("Logging out...")
                        break
                    else:
                        print("Invalid option.")
        elif choice == "3":
            print("Goodbye!")
            db.close()
            break
        else:
            print("Invalid choice.")


if __name__ == "__main__":
    main()



1. Register
2. Login
3. Exit


Choose option:  1
Enter username:  mike


Username already exists.

1. Register
2. Login
3. Exit


Choose option:  2
Enter username:  mike


Logged in as 'mike'

Options:
1. Add task
2. View tasks
3. Mark task done
4. Delete task
5. Update task
6. Filter tasks
7. Sort tasks
8. Filter by date
9. Logout
